The related tutorial online is [here](https://github.com/sithu31296/PyTorch-ONNX-TFLite) with the env:

```shell
tensorflow==2.4.1
onnx==1.8.0
onnx-tf==1.7.0
```

Howerver, based on our test, to successfully convert the model to tensorflow lite, the following requirements are needed:

```shell
tensorflow==2.11.0
onnx==1.14.1
onnx-tf==1.10.0
```

where the onnx-tf can be installed by:
    
```shell
git clone https://github.com/onnx/onnx-tensorflow.git && cd onnx-tensorflow
pip install -e .
```

In [1]:
import numpy as np
import onnxruntime as ort
import onnx
from onnx_tf.backend import prepare
import tensorflow as tf
import pickle


2024-03-07 15:48:49.393933: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-07 15:48:49.594072: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-03-07 15:48:49.594108: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2024-03-07 15:48:50.438204: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2024-

In [2]:
onnx_model_path = "NanoTapor_files/testing_edge_tapor_v12.onnx"
tf_model_path = 'NanoTapor_files/testing_edge_tapor_tf'
tflite_model_path = 'NanoTapor_files/testing_edge_tapor_tf.tflite'
tflite_quant_model_path = 'NanoTapor_files/testing_edge_tapor_tf_quant.tflite'
cali_data_path = 'NanoTapor_files/cali_dataset/x_cal.pkl'

# ONNX to TF

In [3]:
onnx_model = onnx.load(onnx_model_path)
tf_rep = prepare(onnx_model)
tf_rep.export_graph(tf_model_path)

# USING COMMAND LINE !!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# python -m tf2onnx.convert --saved-model edge_tapor_toy_tf --opset 12 --output model.onnx
# https://github.com/onnx/tensorflow-onnx#cli-reference

2024-03-07 15:48:52.273159: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-03-07 15:48:52.273252: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2024-03-07 15:48:52.273299: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2024-03-07 15:48:52.273351: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcufft.so.10'; dlerror: libcufft.so.10: cannot open shared object file: No such file or directory
2024-03-07 15:48:52.378370: W tensorfl

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


INFO:tensorflow:Assets written to: EdgeTapor_files/testing_edge_tapor_tf/assets


INFO:tensorflow:Assets written to: EdgeTapor_files/testing_edge_tapor_tf/assets


TF mdoel inference

In [4]:
tf_model = tf.saved_model.load(tf_model_path)
tf_model.trainable = False
batch_size = 1
channels = 1
height = 24
width = 32
input_tensor = tf.random.uniform([batch_size, channels, height, width])
out = tf_model(**{'thermal_map': input_tensor})

# TF to TF lite (can skip)

In [5]:
# Convert the model
converter = tf.lite.TFLiteConverter.from_saved_model(tf_model_path)
tflite_model = converter.convert()

with open(tflite_model_path, 'wb') as f:
    f.write(tflite_model)

# TFlite to onnx:
# python -m tf2onnx.convert --opset 16 --tflite edge_tapor_toy_tf.tflite --output model_tflt2onnx.onnx
# https://github.com/onnx/tensorflow-onnx#cli-reference

2024-03-07 15:48:56.587736: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2024-03-07 15:48:56.587784: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2024-03-07 15:48:56.588455: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: EdgeTapor_files/testing_edge_tapor_tf
2024-03-07 15:48:56.589173: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2024-03-07 15:48:56.589187: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: EdgeTapor_files/testing_edge_tapor_tf
2024-03-07 15:48:56.597453: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:357] MLIR V1 optimization pass is not enabled
2024-03-07 15:48:56.597894: I tensorflow/cc/saved_model/loader.cc:229] Restoring SavedModel bundle.
2024-03-07 15:48:56.612361: I tensorflow/cc/saved_model/loader.cc:213] Running initialization op on SavedMode

Test the inference of tf lite model

In [6]:
# Load the TFLite model and allocate tensors
interpreter = tf.lite.Interpreter(model_path=tflite_model_path)
interpreter.allocate_tensors()

# Get input and output tensors
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Test the model on random input data
input_shape = input_details[0]['shape']
input_data = np.array(np.random.random_sample(input_shape), dtype=np.float32)
interpreter.set_tensor(input_details[0]['index'], input_data)
interpreter.invoke()
output_data = interpreter.get_tensor(output_details[0]['index'])
print(output_data)

[[-5.4503378e-02  1.3470452e-01  5.8611584e-01 -7.1140967e-02
   6.6913575e-02  5.0813520e-01 -6.0046475e-02  2.1431398e-02
   5.0056887e-01 -4.8851054e-02  4.7152376e-04  4.9844366e-01
  -4.9112290e-02  2.3369812e-03  5.0561678e-01 -8.0019414e-02
   2.1016648e-02  5.0244939e-01 -9.6104048e-02 -5.2059989e-02
   5.2891731e-01 -1.1857785e-01 -1.1511430e-01  5.2633607e-01
  -1.3732748e-01 -1.5091980e-01  5.1979047e-01 -6.6179238e-02
   1.6802894e-02  5.3996491e-01 -6.9964245e-02 -6.3421533e-02
   5.3095233e-01 -8.0199912e-02 -1.2003826e-01  5.2029496e-01
  -8.8223569e-02 -1.4944796e-01  5.0984728e-01 -5.0553188e-02
   3.9346080e-02  5.5174071e-01 -4.2454716e-02 -5.1399237e-03
   5.1394039e-01 -3.3817690e-02  2.7392061e-02  4.8997167e-01
  -4.8188772e-02  4.8625272e-02  4.9969542e-01 -2.9548792e-02
   4.5900710e-02  5.4310453e-01 -3.1528011e-02  5.4170405e-03
   5.2833837e-01 -2.3911757e-02  1.8290281e-02  5.2012628e-01
  -3.8952839e-02  2.5339417e-02  5.2101862e-01]]


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


# TF to TF lite with integer quantization

Integer with float fallback (using default float input/output)
The model is in integer but use float operators when they don't have an integer implementation.

In [7]:
print(tf_model_path)
converter = tf.lite.TFLiteConverter.from_saved_model(tf_model_path)
converter.optimizations = [tf.lite.Optimize.DEFAULT]

with open(cali_data_path, 'rb') as f:
    input = pickle.load(f)

def representative_dataset_gen():
    for i in range(1000):
        temp = input[i]
        yield [temp[np.newaxis,...]]

converter.representative_dataset = representative_dataset_gen
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
tflite_quant_model = converter.convert()
# converter.inference_input_type = tf.int8
# converter.inference_output_type = tf.int8
open(tflite_quant_model_path, "wb").write(tflite_quant_model)

2024-03-07 15:48:56.840060: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2024-03-07 15:48:56.840110: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2024-03-07 15:48:56.840277: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: EdgeTapor_files/testing_edge_tapor_tf
2024-03-07 15:48:56.841070: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2024-03-07 15:48:56.841086: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: EdgeTapor_files/testing_edge_tapor_tf
2024-03-07 15:48:56.842850: I tensorflow/cc/saved_model/loader.cc:229] Restoring SavedModel bundle.
2024-03-07 15:48:56.856285: I tensorflow/cc/saved_model/loader.cc:213] Running initialization op on SavedModel bundle at path: EdgeTapor_files/testing_edge_tapor_tf
2024-03-07 15:48:56.863016: I tensorflow/cc/saved_model/loader.cc:305] SavedM

EdgeTapor_files/testing_edge_tapor_tf


fully_quantize: 0, inference_type: 6, input_inference_type: FLOAT32, output_inference_type: FLOAT32
2024-03-07 15:48:57.314325: I tensorflow/compiler/mlir/lite/flatbuffer_export.cc:2111] Estimated count of arithmetic ops: 1.318 M  ops, equivalently 0.659 M  MACs


165048

# converting the quantized tflite model to hex file

The next step is using the linux command to convert the model into hex file:
```
xxd -i model.tflite > EdgeTapor_files/model_data.cc
```
Then we can use the model_data.cc in the arduino code.

If encounter the error: zsh: command not found: xxd, then we can use the following command to install xxd:
```
sudo apt-get install xxd
```
If the error still exists, then try this in a linux machine locally. OR use sudo:
```
sudo xxd -i model.tflite > EdgeTapor_files/model_data.cc
```
e.g.,
```
sudo xxd -i EdgeTapor_files/testing_edge_tapor_tf_quant.tflite > EdgeTapor_files/testing_model_data.cc
```